## Реализация данной статьи https://habr.com/ru/articles/769124/

In [ ]:
! pip install peft

In [1]:
! pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/10/17/1d00f5ee2577d9c10c238c318b1fe6f6142a989c77277b6163d9549e3f93/langchain-0.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.18 from https://files.pythonhosted.org/packages/bf/b4/1b1b22ab0c57320c5476b735cfe1500e49ddc4425df9e4c2e569e4c4472e/langchain_community-0.0.19-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2,>=0.1.22 from https://files.pythonhosted.org/packages/c8/69/f9cd4ef398973830afe697417070d712ef1fbb3b9a768b8599f555deb687/langchain_core-0.1.22-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1,>=0.0.83 from https://files.pythonhosted.org/packages/71/41/02b

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate
import pandas as pd

ModuleNotFoundError: No module named 'torch'

In [4]:
# Load model from HuggingFace Hub
sent_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
sent_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [5]:
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"

tokenizer = AutoTokenizer.from_pretrained(
              base_model_name,
              trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
device_map = {"": 0}
model = AutoPeftModelForCausalLM.from_pretrained(
              adapt_model_name,
              device_map=device_map,
              torch_dtype=torch.bfloat16)

model = model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

In [6]:
def get_embedding(sentence):
    
    #Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

In [7]:
answers = []
emb_database = torch.empty((0, 384), dtype=torch.float32)

In [9]:
text_qa = pd.read_csv("/data/qa_data/preproc/qa_preproc_wekan.csv", index_col=False)
text_qa.head()

,questions,answer
0,Я посмотрел - с 15 цикла циклы стали по месяцу...,Двухнедельные циклы - следствие некорректной р...
1,Добрый вечер! Добавьте пожалуйста меня - mshul...,В Wekan Вы авторизованы с почты @edu.hse.ru. Д...
2,Здравствуйте! Вопрос от руководителя: как удал...,Добрый день!\nУдалить карточку из Wekan нельзя...
3,Добрый день! Не отображается доска в wekan. Ав...,Возможно возникла ошибка при авторизации в сис...
4,"Добрый вечер, Проблема: не зачисляются часы. П...",Добрый день!\nВам необходимо войти в Wekan чер...


In [10]:
text_qa['qa'] = text_qa['question'] + text_qa['answer']
text_qa['qa']

0    Я посмотрел - с 15 цикла циклы стали по месяцу...
1    Добрый вечер! Добавьте пожалуйста меня - mshul...
2    Здравствуйте! Вопрос от руководителя: как удал...
3    Добрый день! Не отображается доска в wekan. Ав...
4    Добрый вечер, Проблема: не зачисляются часы. П...
5    Здравствуйте, векан пишет: доска не найдена. Д...
6    Добрый вечер. 1) Можете ли добавить @Григорий ...
7    Здравствуйте, я участник проекта 1257! Сегодня...
8    Добрый день, я из проекта 398. Руководитель пр...
Name: q_ans, dtype: object

In [11]:
f = lambda x : ' '.join(x)
text = f(text_qa['qa']) + "{question}\nbot: Вот ответ на ваш вопрос длиной не более 10 слов"

In [12]:
info_prompt_less10 = PromptTemplate.from_template(text)

In [13]:
def get_answer(info_prompt, question):
    
    prompt = info_prompt.format(question=question)   
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), 
                            top_p=0.5,
                            temperature=0.3,
                            attention_mask=inputs["attention_mask"],
                            max_new_tokens=50,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    parsed_answer = output.split("Вот ответ на ваш вопрос длиной не более 10 слов:")[1].strip()

    if "bot:" in parsed_answer:
        parsed_answer = parsed_answer.split("bot:")[0].strip()

    return parsed_answer

In [14]:
question = "Как авторизоваться в векан?" 
emb = get_embedding(question)

In [15]:
def get_cos_sim(question):
    cos_sim = F.cosine_similarity(emb_database, emb, dim=1, eps=1e-8)
    return cos_sim
  
get_cos_sim(question)

tensor([])

In [16]:
answer = get_answer(info_prompt_less10, question)
emb_database = torch.cat((emb_database, emb), 0)
answers.append(answer)
print(f'Answer from model: {answer}')

Answer from model: Авторизуйтесь через ЕЛК.


In [17]:
questions = [
    "Не зачтены часы в карточке в wekan",
    "Не  отображается доска в wekan",
    "Не зачисляются часы в карточке",
    "Руководитель не может проставить часы",
    "Не найдена доска в wekan",
    "Можно ли удалить неверно зачтенную карточку?",
    "Не получается авторизоваться в wekan"
            ] 

In [18]:
for q in questions:
    print(q)
    emb = get_embedding(q)    
    cos_sim = get_cos_sim(q)
    max_value, max_index = torch.max(get_cos_sim(q), dim=0)

    if max_value > 0.9:
        answer = answers[max_index]
        print(f'DATABASE: {answer}')
    else:
        answer = get_answer(info_prompt_less10, q)
        emb_database = torch.cat((emb_database, emb), 0)
        answers.append(answer)
        print(f'MODEL: {answer}')
    print()

Не зачтены часы в карточке в wekan
MODEL: Удалить невозможно, исправить можно.

Вы можете исправить неправильно зачтенную карточку, удалить невозможно.

Вы можете исправить неправильно зачтенную карточку, удалить невозможно.

Не  отображается доска в wekan
MODEL: Авторизуйтесь через ЕЛК.

Не зачисляются часы в карточке
MODEL: Попросите руководителя убрать из названий тех карточек, которые не зачлись, значок кавычек ("). Потом ему нужно будет убрать и заново проставить галочку "Выполнено". После этого часы будут отправлены в кабинет.

Руководитель не может проставить часы
MODEL: "Попросите руководителя убрать из названий тех карточек, которые не зачлись, значок кавычек ("). Потом ему нужно будет убрать и заново проставить галочку "Выполнено". После этого часы будут отправлены в кабинет."

Не найдена доска в wekan
MODEL: Необходимо авторизоваться в Wekan через ЕЛК.

Можно ли удалить неверно зачтенную карточку?
MODEL: нет.

Не получается авторизоваться в wekan
MODEL: Вы не авторизованы в 